In [2]:
import tensorflow as tf
import pickle
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np

In [3]:
# load thhe trained model, scaler pickle, onehot
model = load_model('model.h5')

# load the encoder and scaler
with open('gender_le.pkl', 'rb') as file:
    gender_le = pickle.load(file)

with open('geography_ohe.pkl', 'rb') as file:
    geography_ohe = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [4]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
geo_encoded = geography_ohe.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = geography_ohe.get_feature_names_out(['Geography']))
geo_encoded_df

d:\downloads\codes\DL project\ANN classification\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
# encode catagorical variables
input_df['Gender'] = gender_le.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
input_df = pd.concat([input_df.drop('Geography', axis = 1), geo_encoded_df], axis = 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
input_scaled = scaler.transform(input_df)
input_scaled, input_scaled.shape

(array([[-0.51396295,  0.90192655,  0.08868188, -0.69394774, -0.26384009,
          0.80094036,  0.64071425,  0.97132018, -0.85524227,  1.00260078,
         -0.57795011, -0.57874991]]),
 (1, 12))

In [10]:
#predict churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step


array([[0.05930736]], dtype=float32)

In [11]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.05930736)

In [12]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.


In [13]:
geography_ohe.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [14]:
gender_le.classes_

array(['Female', 'Male'], dtype=object)